In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

In [71]:
import sys
import numpy as np
import pandas as pd

sys.path.append('/lfs/1/sahaana/enrichment/ember/utils')
from preprocessing_utils import compute_BM25, merge_columns, reindex_deepmatcher

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# deepmatcher

In [6]:
path = "/lfs/1/sahaana/enrichment/data/deepmatcher"
merged_col = "merged_all"
separator = "[SEP]"

In [7]:
datasets = {0:"abt_buy_exp_data", 
            1:"amazon_google_exp_data", 
            2:"beer_exp_data", 
            3:"company_exp_data", 
            4:"dblp_acm_exp_data", 
            5:"dblp_scholar_exp_data", 
            6:"dirty_dblp_acm_exp_data", 
            7:"dirty_dblp_scholar_exp_data", 
            8:"dirty_itunes_amazon_exp_data", 
            9:"dirty_walmart_amazon_exp_data", 
            10:"fodors_zagat_exp_data", 
            11:"itunes_amazon_exp_data", 
            12:"walmart_amazon_exp_data"}

## Merge and BM25

In [61]:
for d in datasets:
    query_path = f"{path}/{datasets[d]}/tableA_processed.pkl"
    corpus_path = f"{path}/{datasets[d]}/tableB_processed.pkl"
    
    query_df = pd.read_csv(f"{path}/{datasets[d]}/tableA.csv")
    corpus_df = pd.read_csv(f"{path}/{datasets[d]}/tableB.csv")
    
    query_columns = list(query_df.columns)
    corpus_columns = list(corpus_df.columns)
    
    query_columns.remove('id')
    corpus_columns.remove('id')
    
    query_df = merge_columns(query_df, query_columns, merged_col, query_path, separator)
    corpus_df = merge_columns(corpus_df, corpus_columns, merged_col, corpus_path, separator)
    
    compute_BM25(corpus_df, query_df, "merged_all", datasets[d])

INFO: Pandarallel will run on 56 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Saved abt_buy_exp_data
INFO: Pandarallel will run on 56 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Saved amazon_google_exp_data
INFO: Pandarallel will run on 56 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Saved beer_exp_data
INFO: Pandarallel will run on 56 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Saved company_exp_data
INFO: Pandarallel will run on 56 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Saved dblp_acm_exp_data
INFO: Pandarallel will run on 56 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.
Saved dblp_scholar_exp_

## Reindex Train/Val/Test

In [89]:
for d in datasets:
    l_df = pd.read_csv(f"{path}/{datasets[d]}/tableA.csv")
    r_df = pd.read_csv(f"{path}/{datasets[d]}/tableB.csv")
    
    train = pd.read_csv(f"{path}/{datasets[d]}/train.csv")
    test = pd.read_csv(f"{path}/{datasets[d]}/test.csv")
    val = pd.read_csv(f"{path}/{datasets[d]}/valid.csv")
    
    train_updated = reindex_deepmatcher(l_df, r_df, train)
    test_updated = reindex_deepmatcher(l_df, r_df, test)
    val_updated = reindex_deepmatcher(l_df, r_df, val)
    
    train_updated.to_csv(f"{path}/{datasets[d]}/train_updated.csv", index=False)
    test_updated.to_csv(f"{path}/{datasets[d]}/test_updated.csv", index=False)
    val_updated.to_csv(f"{path}/{datasets[d]}/val_updated.csv", index=False)
    

## Count the num keys in train/test/val, and overlap in train-test train-val val-test

In [33]:
for d in datasets:
    l_df = pd.read_csv(f"{path}/{datasets[d]}/tableA.csv")
    r_df = pd.read_csv(f"{path}/{datasets[d]}/tableB.csv")
    
    train = pd.read_csv(f"{path}/{datasets[d]}/train.csv")
    test = pd.read_csv(f"{path}/{datasets[d]}/test.csv")
    val = pd.read_csv(f"{path}/{datasets[d]}/valid.csv")
    print(datasets[d])
    print(f"{len(l_df)} \t lsize")
    print(f"{len(r_df)} \t rsize")
    print(f"{len(train)} \t train size")
    print(f"{len(val)} \t val size")
    print(f"{len(test)} \t test size")
    print()
    print(f"{train['ltable_id'].nunique()} \t uniques in train ltable")
    print(f"{train['rtable_id'].nunique()} \t uniques in train rtable")

    print(f"{val['ltable_id'].nunique()} \t uniques in val ltable")
    print(f"{val['rtable_id'].nunique()} \t uniques in val rtable")
    
    print(f"{test['ltable_id'].nunique()} \t uniques in test ltable")
    print(f"{test['rtable_id'].nunique()} \t uniques in test rtable")
    print()
    train_val_l = pd.merge(train, val, on='ltable_id', suffixes=('_l','_r'), how='inner')['ltable_id'].nunique()
    train_val_r = pd.merge(train, val, on='rtable_id', suffixes=('_l','_r'), how='inner')['rtable_id'].nunique()
    train_test_l = pd.merge(train, test, on='ltable_id', suffixes=('_l','_r'), how='inner')['ltable_id'].nunique()
    train_test_r = pd.merge(train, test, on='rtable_id', suffixes=('_l','_r'), how='inner')['rtable_id'].nunique()
    val_test_l = pd.merge(val, test, on='ltable_id', suffixes=('_l','_r'), how='inner')['ltable_id'].nunique()
    val_test_r = pd.merge(val, test, on='rtable_id', suffixes=('_l','_r'), how='inner')['rtable_id'].nunique()
    
    print(f"{train_val_l} \t train/val ltable overlap")
    print(f"{train_val_r} \t train/val rtable overlap")
    print(f"{train_test_l} \t train/test ltable overlap")
    print(f"{train_test_r} \t train/test rtable overlap")
    print(f"{val_test_l} \t test/val ltable overlap")
    print(f"{val_test_r} \t test/val rtable overlap")
    
    print()
    print()
    

abt_buy_exp_data
1081 	 lsize
1092 	 rsize
5743 	 train size
1916 	 val size
1916 	 test size

973 	 uniques in train ltable
956 	 uniques in train rtable
728 	 uniques in val ltable
702 	 uniques in val rtable
737 	 uniques in test ltable
700 	 uniques in test rtable

676 	 train/val ltable overlap
649 	 train/val rtable overlap
674 	 train/test ltable overlap
650 	 train/test rtable overlap
547 	 test/val ltable overlap
516 	 test/val rtable overlap


amazon_google_exp_data
1363 	 lsize
3226 	 rsize
6874 	 train size
2293 	 val size
2293 	 test size

1126 	 uniques in train ltable
1788 	 uniques in train rtable
772 	 uniques in val ltable
1107 	 uniques in val rtable
771 	 uniques in test ltable
1090 	 uniques in test rtable

673 	 train/val ltable overlap
903 	 train/val rtable overlap
674 	 train/test ltable overlap
888 	 train/test rtable overlap
531 	 test/val ltable overlap
641 	 test/val rtable overlap


beer_exp_data
4345 	 lsize
3000 	 rsize
268 	 train size
91 	 val size
91 